In [1]:
##################### Section 5.4 (Modelling only)#####################

#Codes for figure are in the file 'Section 6.ipynb'

import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import quandl
import scipy.optimize as sco
import time

path = '/Users/heshan/Desktop/Portfolio Optimization/'
os.chdir(path)

# Import price data
data_aapl = pd.read_csv('AAPL.csv')
data_goog = pd.read_csv('GOOG.csv')
data_market = pd.read_csv('SP500.csv')

datadict = {'AAPL':data_aapl, 'GOOG':data_goog,'Market':data_market}
keylist = ['AAPL','GOOG','Market']
origin_data = pd.DataFrame(index=pd.to_datetime(data_market['Date']),\
                           columns=keylist)
for key in keylist:
    data = datadict[key]
    data.index = pd.to_datetime(data['Date'])
    origin_data[key] = data['Adj Close']
prices = origin_data.copy()

# Calculate returns
returns = prices.pct_change()
returns.dropna(inplace=True)
returns_all = returns.copy()
returns.head()

,AAPL,GOOG,Market
Date,,,
2014-07-15,-0.011716,-0.000154,-0.001932
2014-07-16,-0.005665,-0.003625,0.004201
2014-07-17,-0.017831,-0.015326,-0.011834
2014-07-18,0.014395,0.037213,0.010265
2014-07-21,-0.005189,-0.009427,-0.002320


# Section 5.4 Function of G-learning with linear dynamics

In [2]:
# Get beta in CAPM model
def get_beta(Y,X):
    X = sm.add_constant(X)
    model = sm.OLS(Y,X).fit()
    return model.params[1]

In [3]:
# Function to run the G learning with linear dynamics
def G_learning_with_linear_dynamics(start, end, data, rf, gamma, beta,\
                                    iterations):
    # Locate data
    returns = data.iloc[start:end,]
    T = len(returns.index)
    
    # Set some constants for convenience
    I = np.identity(N)
    In = np.concatenate((I,-I))
    
    # Initialize A_0, A_1, \Sigma, \bar{X}, \bar{a}
    A0t = []
    A1t = []
    sigmaat = []

    xbar = []
    abar = []
    dxt = []
    dat = []

    for t in range(T):
        A0t.append(np.zeros(shape=(2*N,1)))
        A1t.append(np.zeros(shape=(2*N,N)))
        sigmaat.append(np.diag(np.array([1]*(2*N))))

        xbar.append(np.array([[1/N]]*N))
        abar.append(np.array(A0t[0]+np.dot(A1t[0],xbar[t])))
        dxt.append(np.array([[0]]*N))
        dat.append(np.array([[0]]*(2*N)))
    
    # Terminal condition
    duT = -xbar[T-2]-np.dot(In.T,abar[T-1])
    dat[T-1]=np.concatenate([np.maximum(duT,np.array([[0]]*N)),\
                             np.maximum(-duT,np.array([[0]]*N))])
    
    # Initialize market signal z, returns r
    z = np.array([returns['Market']-rf]*N).T
    r = np.array(returns[keylist[0:N]]-rf)
    zt = []
    rt = []
    for t in range(T):
        zt.append((z[t,:].reshape((N,1))))
        rt.append((r[t,:].reshape((N,1))))
    
    # Get beta in CAPM model
    beta_list = np.array([])
    for key in keylist[0:N]:
        beta_list = np.append(beta_list,\
                              get_beta(data.iloc[start:end,][key],\
                              data.iloc[start:end,]['Market']))
    # Make assumptions on W, M
    W = np.diag(beta_list)
    M = np.identity(N)*1e-10
    
    # Error epsilon of CAPM model
    epsilon = r.copy()
    for i in range(N):
        epsilon[:,i] = r[:,i] - beta_list[i]*z[:,i]
    sigmat = np.cov(epsilon.T)
    
    # Further assumptions on the penalty parameters
    lambd = 1
    thetap = np.array([[0.1]]*N)
    thetan = np.array([[0.1]]*N)
    psi = np.array([[0.1]]*N)
    kp = np.array([[0.0]])
    kn = np.array([[0.0]])
    
    # Parameters related to the penalized return
    Raat_tilda = []
    Rxxt_tilda = []
    Raxt_tilda = []
    Rat_tilda = []
    Rxt_tilda = []
    for t in range(T):
        Raat_tilda.append\
            (np.concatenate((np.concatenate((-M-lambd*sigmat,\
                                             M+lambd*sigmat),axis=1),\
                             np.concatenate((M+lambd*sigmat,\
                                             -M-lambd*sigmat),axis=1))\
                            ,axis=0))
        Rxxt_tilda.append(-lambd*sigmat)
        Raxt_tilda.append(np.concatenate((-M-2*lambd*sigmat-thetap,\
                                          M+2*lambd*sigmat-thetan),\
                                         axis=0))
        Rat_tilda.append(np.concatenate((np.dot(W,zt[t])-\
                                         kp,np.dot(W,zt[t])-kn),\
                                        axis=0))
        Rxt_tilda.append(np.linalg.multi_dot([W-psi,zt[t]]))
    
    # Initilize parameters for parametrizations
    Dt = [0]*T
    Ht = [0]*T
    ft = [0]*T

    D2t = [0]*T
    H2t = [0]*T
    f2t = [0]*T

    At_hat = [0]*T
    Bt_hat = [0]*T
    Ct_hat = [0]*T

    Faat = [0]*T
    Fxxt = [0]*T
    Faxt = [0]*T
    Fat = [0]*T
    Fxt = [0]*T
    ft_hat = [0]*T

    Gaat = [0]*T
    Gxxt = [0]*T
    Gaxt = [0]*T
    Gat = [0]*T
    Gxt = [0]*T
    gt = [0]*T

    sigmaa_tilde = sigmaat.copy()
    
    # Run the loop many times
    for i in range(iterations): 

        print (beta)

        # Parametrization for R
        Raat = Raat_tilda.copy()
        Rxxt = Rxxt_tilda.copy()
        Raxt = Raxt_tilda.copy()
        Rat = []
        Rxt = []
        Rt = []
        
        for t in range(T): # (cont.) Parametrization of R
            Rat.append(Rat_tilda[t]+2*np.dot(Raat_tilda[t],abar[t])+\
                       np.dot(Raxt_tilda[t],xbar[t]))
            Rxt.append(Rxt_tilda[t]+2*np.dot(Rxxt_tilda[t],xbar[t])+\
                       np.dot(Raxt_tilda[t].T,abar[t]))
            Rt.append(np.linalg.multi_dot([abar[t].T,Raat_tilda[t],\
                                           abar[t]])+\
                      np.linalg.multi_dot([xbar[t].T,Rxxt_tilda[t],\
                                           xbar[t]])+\
                      np.linalg.multi_dot([abar[t].T,Raxt_tilda[t],\
                                           xbar[t]])+\
                      np.dot(abar[t].T,Rat_tilda[t])+\
                      np.dot(xbar[t].T,Rxt_tilda[t]))
        
        # Parameters at time T
        Dt[T-1] = Rxxt[T-1]
        Ht[T-1] = Rxt[T-1]+np.dot(Raxt[T-1].T,dat[T-1])
        ft[T-1] = np.linalg.multi_dot([dat[T-1].T,Raat[T-1],\
                                       dat[T-1]])+\
                  np.dot(dat[T-1].T,Rat[T-1])+Rt[T-1]
        
        # Backward iteration to update parametrization of 
        # G, F, E[F_{t+1}]
        for t in range(T-2,-1,-1):

            At_hat[t] = np.diag(np.diag(I+rf*I+np.dot(W,zt[t])-\
                                    np.dot(M.T,np.dot(In.T,abar[t]))))
            Bt_hat[t] = At_hat[t] - np.multiply((xbar[t]+\
                                             np.dot(In.T,abar[t])),M)
            Ct_hat[t] = np.dot(At_hat[t],xbar[t]+np.dot(In.T,abar[t]))\
                        -xbar[t+1]

            # Parametrization for E[F_{t+1}]
            Faat[t] = np.linalg.multi_dot([In, Bt_hat[t].T, Dt[t+1],\
                                           Bt_hat[t], In.T])+\
                      np.linalg.multi_dot([In,np.multiply(Dt[t+1],\
                                                      sigmat),In.T])
            Fxxt[t] = np.linalg.multi_dot([At_hat[t].T, Dt[t+1], \
                                           At_hat[t]])+\
                      np.multiply(Dt[t+1],sigmat)
            Faxt[t] = 2*np.linalg.multi_dot([In, Bt_hat[t].T,Dt[t+1],\
                                             At_hat[t]])+\
                      2*np.linalg.multi_dot([In,np.multiply(Dt[t+1],\
                                                            sigmat)])
            Fat[t] = np.linalg.multi_dot([In, Bt_hat[t].T, Ht[t+1]])+\
                     2*np.linalg.multi_dot([In, Bt_hat[t].T, Dt[t+1],\
                                            Ct_hat[t]])+\
                     2*np.linalg.multi_dot([In,np.multiply(Dt[t+1],\
                                                           sigmat),\
                                    (xbar[t]+np.dot(In.T,abar[t]))])
            Fxt[t] = np.linalg.multi_dot([At_hat[t].T, Ht[t+1]])+\
                     2*np.linalg.multi_dot([At_hat[t].T, Dt[t+1], \
                                            Ct_hat[t]])+\
                     2*np.linalg.multi_dot([np.multiply(Dt[t+1],\
                                                        sigmat),\
                                       (xbar[t]+np.dot(In.T,abar[t]))])
            ft_hat[t] = ft[t+1]+np.dot(Ct_hat[t].T,Ht[t+1])+\
                np.linalg.multi_dot([Ct_hat[t].T, Dt[t+1],Ct_hat[t]])+\
                np.linalg.multi_dot([(xbar[t]+\
                                      np.dot(In.T,abar[t])).T,\
                                     np.multiply(Dt[t+1],sigmat),\
                                     (xbar[t]+np.dot(In.T,abar[t]))])

            # Parametrization for G_t
            Gaat[t] = Raat[t]+gamma*Faat[t]
            Gxxt[t] = Rxxt[t]+gamma*Fxxt[t]
            Gaxt[t] = Raxt[t]+gamma*Faxt[t]
            Gat[t] = Rat[t]+gamma*Fat[t]
            Gxt[t] = Rxt[t]+gamma*Fxt[t]
            gt[t] = Rt[t]+gamma*ft_hat[t]

            dat_tilde = A0t[t]+np.dot(A1t[t],xbar[t]) - abar[t]
            sigmaa_tilde[t] = np.linalg.inv(sigmaat[t])-2*beta*Gaat[t]
            
            # Eigenvalue method to slove the problem of 
            # matrix inconvertibility
            max_egvalue = np.linalg.eig(Gaat[t])[0][0]
            count = 0
            while (np.linalg.det(sigmaa_tilde[t])<=0):
                if (count==0):
                    beta = abs(1/max_egvalue)
                else:
                    beta = abs(beta/max_egvalue)

                if (beta<1e-50):
                    beta = 0           
                sigmaa_tilde[t] = np.linalg.inv(sigmaat[t])-\
                                  2*beta*Gaat[t]
                count +=1

            # Parametrization for F_t
            bt = abar[t]-A0t[t]-np.dot(A1t[t],xbar[t])
            
            if beta == 0: # Special case for beta is zero
                tau = np.diag(np.array([1]*(2*N)))
                gammab = np.diag(np.array([0]*(2*N)))
                Lb = 0
            else: 
                tau = 1/beta*(np.linalg.inv(sigmaat[t])-\
                   np.linalg.multi_dot([np.linalg.inv(sigmaat[t]).T,\
                                       np.linalg.inv(sigmaa_tilde[t]),\
                                       np.linalg.inv(sigmaat[t])]))
                gammab = np.dot(np.linalg.inv(sigmaa_tilde[t]),\
                                np.linalg.inv(sigmaat[t]))
                Lb=1/(2*beta)*(np.log(np.linalg.det(sigmaat[t]))+\
                               np.log(np.linalg.det(sigmaa_tilde[t])))

            Eax = np.dot(gammab,A1t[t])+\
                  0.5*beta*np.dot(np.linalg.inv(sigmaa_tilde[t]),\
                                  Gaxt[t])
            Dax = np.dot(Gaxt[t].T,gammab)-np.dot(A1t[t].T,tau)
            Ea = np.linalg.multi_dot([A1t[t].T,gammab,Gat[t]])+\
                 beta*np.linalg.multi_dot([Gaxt[t].T,\
                                       np.linalg.inv(sigmaa_tilde[t]),\
                                       Gat[t]])
            
            # Parametrization for F_t (cont.)
            Dt[t] = Gxxt[t] + np.dot(Gaxt[t].T,Eax) - \
                    0.5*np.linalg.multi_dot([A1t[t].T, tau, A1t[t]])
            Ht[t] = Gxt[t] - np.dot(Dax,bt) + \
                    np.linalg.multi_dot([A1t[t].T,gammab,Gat[t]]) + \
                    beta*np.linalg.multi_dot([Gaxt[t].T,\
                              np.linalg.inv(sigmaa_tilde[t]),Gat[t]])
            ft[t] = gt[t] - 0.5*np.linalg.multi_dot([bt.T,tau,bt]) -\
                    np.linalg.multi_dot([Gat[t].T,gammab,bt])+\
                    beta/2*np.linalg.multi_dot([Gat[t].T,\
                            np.linalg.inv(sigmaa_tilde[t]),Gat[t]])-Lb

            # Update of mean and variance
            sigmaa2 = np.linalg.inv(sigmaa_tilde[t])
            A0t[t] = abar[t] + np.linalg.multi_dot([sigmaa2, \
                                            np.linalg.inv(sigmaat[t]),\
                                            A0t[t]-abar[t]]) + \
                                            beta*np.dot(sigmaa2,\
                                              Gat[t]-\
                                              np.dot(Gaxt[t],xbar[t]))
            A1t[t] = np.dot(sigmaa2,(np.dot(np.linalg.inv(sigmaat[t]),\
                                            A1t[t])+beta*Gaxt[t]))
            sigmaat[t] = sigmaa2

        # Forward iteration to update trajectories
        for t in range(0,T-1):
            abar[t] = A0t[t]+np.dot(A1t[t], xbar[t])

        for t in range(0,T-1):
            xbar[t+1] = np.multiply(1+rt[t],xbar[t]+\
                                    np.dot(In.T,abar[t]))
        for t in range(0,T-1):
            abar[t] = 0*abar[t]
        # Set the terminal condition again
        duT = -xbar[T-2]-np.dot(In.T,abar[T-1])
        dat[T-1]=np.concatenate([np.maximum(duT,np.array([[0]]*N)),\
                                 np.maximum(-duT,np.array([[0]]*N))])

        print (A0t[t],A1t[t])
        
    return (A0t,A1t)

# Model training

In [4]:
# Set some parameters
train_window = 63
T_all=prices.shape[0]-train_window-1
N=prices.shape[1]-1 #exclude market portfolio
rf = (1+0.02)**(1/365)-1 #Risk-free rate
gamma = 0.01
beta = 0.01
iterations = 100

In [5]:
# Tune parameters through training dataset
# First 70% data as traning set
training_data_index = int(len(returns_all.index)*0.7)
data_train = returns_all.iloc[:training_data_index,]
data_test = returns_all.iloc[training_data_index:,]
A0_list_train = []
A1_list_train = []
for i in range(int(len(data_train.index)/train_window)-1):
    start = i * train_window
    end = start + train_window
    # Run the model
    A0, A1 = G_learning_with_linear_dynamics(start, end, data_train,\
                                             rf, gamma, beta,\
                                             iterations)
    A0_list_train.append(A0)
    A1_list_train.append(A1)

/Users/heshan/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


0.01
[[-0.00018744]
 [-0.00028836]
 [-0.00016634]
 [-0.00026637]] [[-0.00100251 -0.00099994]
 [-0.00099994 -0.00100084]
 [-0.00099749 -0.00100006]
 [-0.00100006 -0.00099916]]
0.01
[[-0.00037499]
 [-0.00057682]
 [-0.00033257]
 [-0.00053263]] [[-0.00200503 -0.00199989]
 [-0.00199989 -0.00200167]
 [-0.00199497 -0.00200011]
 [-0.00200011 -0.00199833]]
0.01
[[-0.00056253]
 [-0.00086529]
 [-0.0004988 ]
 [-0.0007989 ]] [[-0.00300754 -0.00299983]
 [-0.00299983 -0.00300251]
 [-0.00299246 -0.00300017]
 [-0.00300017 -0.00299749]]
0.01
[[-0.00075007]
 [-0.00115375]
 [-0.00066503]
 [-0.00106517]] [[-0.00401005 -0.00399978]
 [-0.00399978 -0.00400335]
 [-0.00398995 -0.00400022]
 [-0.00400022 -0.00399665]]
0.01
[[-0.00093761]
 [-0.00144221]
 [-0.00083127]
 [-0.00133143]] [[-0.00501257 -0.00499972]
 [-0.00499972 -0.00500419]
 [-0.00498743 -0.00500028]
 [-0.00500028 -0.00499581]]
0.01
[[-0.00112515]
 [-0.00173067]
 [-0.0009975 ]
 [-0.0015977 ]] [[-0.00601508 -0.00599967]
 [-0.00599967 -0.00600502]
 [-0.

[[-0.00900326]
 [-0.01384741]
 [-0.00797801]
 [-0.01277956]] [[-0.0481206  -0.04799735]
 [-0.04799735 -0.04804019]
 [-0.0478794  -0.04800265]
 [-0.04800265 -0.04795981]]
0.01
[[-0.00919093]
 [-0.014136  ]
 [-0.00814411]
 [-0.01304569]] [[-0.04912312 -0.04899729]
 [-0.04899729 -0.04904103]
 [-0.04887688 -0.04900271]
 [-0.04900271 -0.04895897]]
0.01
[[-0.00937861]
 [-0.0144246 ]
 [-0.00831021]
 [-0.01331182]] [[-0.05012563 -0.04999724]
 [-0.04999724 -0.05004187]
 [-0.04987437 -0.05000276]
 [-0.05000276 -0.04995813]]
0.01
[[-0.0095663 ]
 [-0.0147132 ]
 [-0.0084763 ]
 [-0.01357795]] [[-0.05112814 -0.05099718]
 [-0.05099718 -0.05104271]
 [-0.05087186 -0.05100282]
 [-0.05100282 -0.05095729]]
0.01
[[-0.00975399]
 [-0.01500182]
 [-0.00864239]
 [-0.01384406]] [[-0.05213065 -0.05199713]
 [-0.05199713 -0.05204354]
 [-0.05186935 -0.05200287]
 [-0.05200287 -0.05195646]]
0.01
[[-0.00994169]
 [-0.01529044]
 [-0.00880846]
 [-0.01411017]] [[-0.05313316 -0.05299707]
 [-0.05299707 -0.05304438]
 [-0.05286

[[-0.01821175]
 [-0.02800096]
 [-0.01610456]
 [-0.02580769]] [[-0.09724366 -0.09699464]
 [-0.09699464 -0.09708122]
 [-0.09675634 -0.09700536]
 [-0.09700536 -0.09691878]]
0.01
[[-0.01840004]
 [-0.02829017]
 [-0.01627005]
 [-0.02607321]] [[-0.09824617 -0.09799458]
 [-0.09799458 -0.09808206]
 [-0.09775383 -0.09800542]
 [-0.09800542 -0.09791794]]
0.01
[[-0.01858835]
 [-0.02857939]
 [-0.01643552]
 [-0.02633872]] [[-0.09924868 -0.09899453]
 [-0.09899453 -0.09908289]
 [-0.09875132 -0.09900547]
 [-0.09900547 -0.09891711]]
0.01
[[-0.01877667]
 [-0.02886864]
 [-0.01660098]
 [-0.0266042 ]] [[-0.10025119 -0.09999447]
 [-0.09999447 -0.10008373]
 [-0.09974881 -0.10000553]
 [-0.10000553 -0.09991627]]
0.01
[[-1.21218052e-04]
 [-7.73639628e-05]
 [-9.96735856e-05]
 [-5.65605444e-05]] [[-0.00100168 -0.00100014]
 [-0.00100014 -0.00100204]
 [-0.00099832 -0.00099986]
 [-0.00099986 -0.00099796]]
0.01
[[-0.00024259]
 [-0.00015489]
 [-0.00019919]
 [-0.00011296]] [[-0.00200337 -0.00200028]
 [-0.00200028 -0.0020

[[-0.00558392]
 [-0.00356672]
 [-0.0045771 ]
 [-0.0025938 ]] [[-0.0460774  -0.04600642]
 [-0.04600642 -0.04609396]
 [-0.0459226  -0.04599358]
 [-0.04599358 -0.04590604]]
0.01
[[-0.0057054 ]
 [-0.00364435]
 [-0.00467651]
 [-0.0026501 ]] [[-0.04707909 -0.04700656]
 [-0.04700656 -0.047096  ]
 [-0.04692091 -0.04699344]
 [-0.04699344 -0.046904  ]]
0.01
[[-0.00582689]
 [-0.00372199]
 [-0.00477591]
 [-0.00270639]] [[-0.04808077 -0.0480067 ]
 [-0.0480067  -0.04809805]
 [-0.04791923 -0.0479933 ]
 [-0.0479933  -0.04790195]]
0.01
[[-0.00594838]
 [-0.00379963]
 [-0.00487531]
 [-0.00276267]] [[-0.04908245 -0.04900684]
 [-0.04900684 -0.04910009]
 [-0.04891755 -0.04899316]
 [-0.04899316 -0.04889991]]
0.01
[[-0.00606989]
 [-0.00387729]
 [-0.00497469]
 [-0.00281894]] [[-0.05008413 -0.05000697]
 [-0.05000697 -0.05010213]
 [-0.04991587 -0.04999303]
 [-0.04999303 -0.04989787]]
0.01
[[-0.0061914 ]
 [-0.00395494]
 [-0.00507408]
 [-0.00287521]] [[-0.05108582 -0.05100711]
 [-0.05100711 -0.05110417]
 [-0.05091

...

[[-2.55454676e-05]
 [-2.48240748e-05]
 [-4.96214199e-06]
 [-4.26707137e-06]] [[-0.00100158 -0.001     ]
 [-0.001      -0.00100073]
 [-0.00099842 -0.001     ]
 [-0.001      -0.00099927]]
0.01
[[-5.21577337e-05]
 [-5.07165379e-05]
 [-8.85748548e-06]
 [-7.46575447e-06]] [[-0.00200316 -0.002     ]
 [-0.002      -0.00200146]
 [-0.00199684 -0.002     ]
 [-0.002      -0.00199854]]
0.01
[[-7.87686415e-05]
 [-7.66076801e-05]
 [-1.27541872e-05]
 [-1.06657585e-05]] [[-0.00300474 -0.003     ]
 [-0.003      -0.00300218]
 [-0.00299526 -0.003     ]
 [-0.003      -0.00299782]]
0.01
[[-1.05378529e-04]
 [-1.02497840e-04]
 [-1.66519091e-05]
 [-1.38667453e-05]] [[-0.00400633 -0.004     ]
 [-0.004      -0.00400291]
 [-0.00399367 -0.004     ]
 [-0.004      -0.00399709]]
0.01
[[-1.31987613e-04]
 [-1.28387233e-04]
 [-2.05504347e-05]
 [-1.70684980e-05]] [[-0.00500791 -0.005     ]
 [-0.005      -0.00500364]
 [-0.00499209 -0.005     ]
 [-0.005      -0.00499636]]
0.01
[[-1.58596110e-04]
 [-1.54276078e-04]
 [-2.44

[[-0.00119819]
 [-0.00116583]
 [-0.00017466]
 [-0.00014327]] [[-0.04507115 -0.04499996]
 [-0.04499996 -0.04503276]
 [-0.04492885 -0.04500004]
 [-0.04500004 -0.04496724]]
0.01
[[-0.00122495]
 [-0.00119188]
 [-0.0001784 ]
 [-0.00014632]] [[-0.04607273 -0.04599996]
 [-0.04599996 -0.04603349]
 [-0.04592727 -0.04600004]
 [-0.04600004 -0.04596651]]
0.01
[[-0.00125172]
 [-0.00121793]
 [-0.00018214]
 [-0.00014935]] [[-0.04707431 -0.04699996]
 [-0.04699996 -0.04703422]
 [-0.04692569 -0.04700004]
 [-0.04700004 -0.04696578]]
0.01
[[-0.0012785 ]
 [-0.00124399]
 [-0.00018586]
 [-0.00015238]] [[-0.0480759  -0.04799996]
 [-0.04799996 -0.04803495]
 [-0.0479241  -0.04800004]
 [-0.04800004 -0.04796505]]
0.01
[[-0.00130529]
 [-0.00127007]
 [-0.00018958]
 [-0.0001554 ]] [[-0.04907748 -0.04899996]
 [-0.04899996 -0.04903567]
 [-0.04892252 -0.04900004]
 [-0.04900004 -0.04896433]]
0.01
[[-0.00133209]
 [-0.00129615]
 [-0.00019329]
 [-0.00015841]] [[-0.05007906 -0.04999996]
 [-0.04999996 -0.0500364 ]
 [-0.04992

[[-0.00249638]
 [-0.00242961]
 [-0.00034083]
 [-0.00027587]] [[-0.09314703 -0.09299993]
 [-0.09299993 -0.0930677 ]
 [-0.09285297 -0.09300007]
 [-0.09300007 -0.0929323 ]]
0.01
[[-0.00252381]
 [-0.00245633]
 [-0.0003439 ]
 [-0.00027824]] [[-0.09414861 -0.09399992]
 [-0.09399992 -0.09406843]
 [-0.09385139 -0.09400008]
 [-0.09400008 -0.09393157]]
0.01
[[-0.00255127]
 [-0.00248307]
 [-0.00034696]
 [-0.00028059]] [[-0.09515019 -0.09499992]
 [-0.09499992 -0.09506916]
 [-0.09484981 -0.09500008]
 [-0.09500008 -0.09493084]]
0.01
[[-0.00257874]
 [-0.00250983]
 [-0.00034999]
 [-0.00028292]] [[-0.09615177 -0.09599992]
 [-0.09599992 -0.09606989]
 [-0.09584823 -0.09600008]
 [-0.09600008 -0.09593011]]
0.01
[[-0.00260624]
 [-0.00253661]
 [-0.000353  ]
 [-0.00028523]] [[-0.09715335 -0.09699992]
 [-0.09699992 -0.09707061]
 [-0.09684665 -0.09700008]
 [-0.09700008 -0.09692939]]
0.01
[[-0.00263375]
 [-0.00256341]
 [-0.00035599]
 [-0.00028752]] [[-0.09815493 -0.09799992]
 [-0.09799992 -0.09807134]
 [-0.09784

[[0.0039274 ]
 [0.0039115 ]
 [0.00479663]
 [0.00478075]] [[-0.04107255 -0.04103224]
 [-0.04103224 -0.04107241]
 [-0.04092745 -0.04096776]
 [-0.04096776 -0.04092759]]
0.01
[[0.00402312]
 [0.00400684]
 [0.0049137 ]
 [0.00489742]] [[-0.04207432 -0.04203302]
 [-0.04203302 -0.04207418]
 [-0.04192568 -0.04196698]
 [-0.04196698 -0.04192582]]
0.01
[[0.00411883]
 [0.00410216]
 [0.00503076]
 [0.0050141 ]] [[-0.04307609 -0.04303381]
 [-0.04303381 -0.04307594]
 [-0.04292391 -0.04296619]
 [-0.04296619 -0.04292406]]
0.01
[[0.00421454]
 [0.00419748]
 [0.00514784]
 [0.00513079]] [[-0.04407786 -0.0440346 ]
 [-0.0440346  -0.04407771]
 [-0.04392214 -0.0439654 ]
 [-0.0439654  -0.04392229]]
0.01
[[0.00431024]
 [0.00429279]
 [0.00526492]
 [0.00524748]] [[-0.04507963 -0.04503538]
 [-0.04503538 -0.04507947]
 [-0.04492037 -0.04496462]
 [-0.04496462 -0.04492053]]
0.01
[[0.00440594]
 [0.0043881 ]
 [0.005382  ]
 [0.00536418]] [[-0.0460814  -0.04603617]
 [-0.04603617 -0.04608124]
 [-0.0459186  -0.04596383]
 [-0.04

[[0.00851179]
 [0.00847729]
 [0.01042574]
 [0.01039126]] [[-0.08915746 -0.08906995]
 [-0.08906995 -0.08915715]
 [-0.08884254 -0.08893005]
 [-0.08893005 -0.08884285]]
0.01
[[0.00860699]
 [0.0085721 ]
 [0.01054332]
 [0.01050845]] [[-0.09015923 -0.09007074]
 [-0.09007074 -0.09015892]
 [-0.08984077 -0.08992926]
 [-0.08992926 -0.08984108]]
0.01
[[0.00870217]
 [0.0086669 ]
 [0.01066092]
 [0.01062567]] [[-0.091161   -0.09107153]
 [-0.09107153 -0.09116068]
 [-0.090839   -0.09092847]
 [-0.09092847 -0.09083932]]
0.01
[[0.00879734]
 [0.00876167]
 [0.01077854]
 [0.01074289]] [[-0.09216277 -0.09207231]
 [-0.09207231 -0.09216245]
 [-0.09183723 -0.09192769]
 [-0.09192769 -0.09183755]]
0.01
[[0.00889249]
 [0.00885644]
 [0.01089617]
 [0.01086014]] [[-0.09316454 -0.0930731 ]
 [-0.0930731  -0.09316421]
 [-0.09283546 -0.0929269 ]
 [-0.0929269  -0.09283579]]
0.01
[[0.00898762]
 [0.00895118]
 [0.01101382]
 [0.0109774 ]] [[-0.09416631 -0.09407388]
 [-0.09407388 -0.09416598]
 [-0.09383369 -0.09392612]
 [-0.09

In [6]:
# Train the model on testing dataset
A0_list_test = []
A1_list_test = []
for i in range(int(len(data_test.index)/train_window)-1):
    start = i * train_window
    end = start + train_window
    # Run the model
    A0, A1 = G_learning_with_linear_dynamics(start, end, data_test, \
                                             rf, gamma, beta, \
                                             iterations)
    A0_list_test.append(A0)
    A1_list_test.append(A1)

0.01
[[0.00016618]
 [0.00021483]
 [0.00018749]
 [0.00023658]] [[-0.0010021  -0.00100029]
 [-0.00100029 -0.00100156]
 [-0.0009979  -0.00099971]
 [-0.00099971 -0.00099844]]
0.01
[[0.00033291]
 [0.00043023]
 [0.00037441]
 [0.00047261]] [[-0.0020042  -0.00200057]
 [-0.00200057 -0.00200311]
 [-0.0019958  -0.00199943]
 [-0.00199943 -0.00199689]]
0.01
[[0.00049965]
 [0.00064562]
 [0.00056134]
 [0.00070863]] [[-0.00300629 -0.00300086]
 [-0.00300086 -0.00300467]
 [-0.00299371 -0.00299914]
 [-0.00299914 -0.00299533]]
0.01
[[0.00066639]
 [0.00086102]
 [0.00074827]
 [0.00094465]] [[-0.00400839 -0.00400115]
 [-0.00400115 -0.00400623]
 [-0.00399161 -0.00399885]
 [-0.00399885 -0.00399377]]
0.01
[[0.00083313]
 [0.00107642]
 [0.00093519]
 [0.00118067]] [[-0.00501049 -0.00500143]
 [-0.00500143 -0.00500778]
 [-0.00498951 -0.00499857]
 [-0.00499857 -0.00499222]]
0.01
[[0.00099987]
 [0.00129182]
 [0.00112211]
 [0.00141669]] [[-0.00601259 -0.00600172]
 [-0.00600172 -0.00600934]
 [-0.00598741 -0.00599828]
 [

[[0.00866872]
 [0.01119892]
 [0.00972177]
 [0.0122748 ]] [[-0.05210907 -0.05201488]
 [-0.05201488 -0.05208093]
 [-0.05189093 -0.05198512]
 [-0.05198512 -0.05191907]]
0.01
[[0.00883534]
 [0.01141419]
 [0.00990882]
 [0.01251095]] [[-0.05311116 -0.05301517]
 [-0.05301517 -0.05308248]
 [-0.05288884 -0.05298483]
 [-0.05298483 -0.05291752]]
0.01
[[0.00900194]
 [0.01162945]
 [0.01009588]
 [0.0127471 ]] [[-0.05411326 -0.05401545]
 [-0.05401545 -0.05408404]
 [-0.05388674 -0.05398455]
 [-0.05398455 -0.05391596]]
0.01
[[0.00916854]
 [0.01184471]
 [0.01028294]
 [0.01298326]] [[-0.05511536 -0.05501574]
 [-0.05501574 -0.05508559]
 [-0.05488464 -0.05498426]
 [-0.05498426 -0.05491441]]
0.01
[[0.00933513]
 [0.01205996]
 [0.01047002]
 [0.01321943]] [[-0.05611745 -0.05601603]
 [-0.05601603 -0.05608715]
 [-0.05588255 -0.05598397]
 [-0.05598397 -0.05591285]]
0.01
[[0.00950172]
 [0.0122752 ]
 [0.0106571 ]
 [0.01345561]] [[-0.05711955 -0.05701631]
 [-0.05701631 -0.05708871]
 [-0.05688045 -0.05698369]
 [-0.05

[[0.01665429]
 [0.02151991]
 [0.01871205]
 [0.02362185]] [[-0.1002097  -0.10002861]
 [-0.10002861 -0.1001556 ]
 [-0.0997903  -0.09997139]
 [-0.09997139 -0.0998444 ]]
0.01
[[2.51011734e-05]
 [4.61886355e-05]
 [4.63580139e-05]
 [6.84074260e-05]] [[-0.00100257 -0.00100025]
 [-0.00100025 -0.00100166]
 [-0.00099743 -0.00099975]
 [-0.00099975 -0.00099834]]
0.01
[[4.90348658e-05]
 [9.12069143e-05]
 [9.38835088e-05]
 [1.37985209e-04]] [[-0.00200515 -0.00200051]
 [-0.00200051 -0.00200332]
 [-0.00199485 -0.00199949]
 [-0.00199949 -0.00199668]]
0.01
[[7.29715515e-05]
 [1.36228155e-04]
 [1.41406010e-04]
 [2.07560030e-04]] [[-0.00300772 -0.00300076]
 [-0.00300076 -0.00300498]
 [-0.00299228 -0.00299924]
 [-0.00299924 -0.00299502]]
0.01
[[9.69108278e-05]
 [1.81251953e-04]
 [1.88925921e-04]
 [2.77132293e-04]] [[-0.0040103  -0.00400101]
 [-0.00400101 -0.00400664]
 [-0.0039897  -0.00399899]
 [-0.00399899 -0.00399336]]
0.01
[[0.00012085]
 [0.00022628]
 [0.00023644]
 [0.0003467 ]] [[-0.00501287 -0.0050012

[[0.00114948]
 [0.00216149]
 [0.00228057]
 [0.00333912]] [[-0.04812354 -0.04801214]
 [-0.04801214 -0.04807964]
 [-0.04787646 -0.04798786]
 [-0.04798786 -0.04792036]]
0.01
[[0.0011733 ]
 [0.00220641]
 [0.0023282 ]
 [0.0034088 ]] [[-0.04912611 -0.04901239]
 [-0.04901239 -0.0490813 ]
 [-0.04887389 -0.04898761]
 [-0.04898761 -0.0489187 ]]
0.01
[[0.00119713]
 [0.00225131]
 [0.00237583]
 [0.00347849]] [[-0.05012869 -0.05001264]
 [-0.05001264 -0.05008296]
 [-0.04987131 -0.04998736]
 [-0.04998736 -0.04991704]]
0.01
[[0.00122094]
 [0.0022962 ]
 [0.00242348]
 [0.00354819]] [[-0.05113126 -0.0510129 ]
 [-0.0510129  -0.05108462]
 [-0.05086874 -0.0509871 ]
 [-0.0509871  -0.05091538]]
0.01
[[0.00124474]
 [0.00234109]
 [0.00247113]
 [0.0036179 ]] [[-0.05213383 -0.05201315]
 [-0.05201315 -0.05208628]
 [-0.05186617 -0.05198685]
 [-0.05198685 -0.05191372]]
0.01
[[0.00126854]
 [0.00238597]
 [0.0025188 ]
 [0.00368762]] [[-0.0531364  -0.0530134 ]
 [-0.0530134  -0.05308794]
 [-0.0528636  -0.0529866 ]
 [-0.05

[[0.0022807 ]
 [0.00430459]
 [0.00457939]
 [0.00669663]] [[-0.09624702 -0.09602426]
 [-0.09602426 -0.09615926]
 [-0.09575298 -0.09597574]
 [-0.09597574 -0.09584074]]
0.01
[[0.00230389]
 [0.00434887]
 [0.00462765]
 [0.00676695]] [[-0.09724959 -0.09702452]
 [-0.09702452 -0.09716092]
 [-0.09675041 -0.09697548]
 [-0.09697548 -0.09683908]]
0.01
[[0.00232708]
 [0.00439313]
 [0.00467592]
 [0.00683728]] [[-0.09825216 -0.09802477]
 [-0.09802477 -0.09816258]
 [-0.09774784 -0.09797523]
 [-0.09797523 -0.09783742]]
0.01
[[0.00235024]
 [0.00443737]
 [0.00472422]
 [0.00690764]] [[-0.09925473 -0.09902502]
 [-0.09902502 -0.09916424]
 [-0.09874527 -0.09897498]
 [-0.09897498 -0.09883576]]
0.01
[[0.00237338]
 [0.00448159]
 [0.00477254]
 [0.00697802]] [[-0.1002573  -0.10002528]
 [-0.10002528 -0.10016589]
 [-0.0997427  -0.09997472]
 [-0.09997472 -0.09983411]]
0.01
[[-1.57938576e-05]
 [-1.90012164e-05]
 [ 4.47425911e-06]
 [ 1.47889255e-06]] [[-0.00100296 -0.0010004 ]
 [-0.0010004  -0.00100143]
 [-0.00099704 

...

[[-0.00148439]
 [-0.00176708]
 [ 0.00048826]
 [ 0.00022512]] [[-0.0882604  -0.08803556]
 [-0.08803556 -0.08812545]
 [-0.0877396  -0.08796444]
 [-0.08796444 -0.08787455]]
0.01
[[-0.00150168]
 [-0.00178759]
 [ 0.00049424]
 [ 0.0002281 ]] [[-0.08926336 -0.08903596]
 [-0.08903596 -0.08912687]
 [-0.08873664 -0.08896404]
 [-0.08896404 -0.08887313]]
0.01
[[-0.00151899]
 [-0.00180811]
 [ 0.00050022]
 [ 0.0002311 ]] [[-0.09026631 -0.09003637]
 [-0.09003637 -0.0901283 ]
 [-0.08973369 -0.08996363]
 [-0.08996363 -0.0898717 ]]
0.01
[[-0.00153631]
 [-0.00182865]
 [ 0.00050623]
 [ 0.00023412]] [[-0.09126927 -0.09103677]
 [-0.09103677 -0.09112972]
 [-0.09073073 -0.09096323]
 [-0.09096323 -0.09087028]]
0.01
[[-0.00155365]
 [-0.00184921]
 [ 0.00051225]
 [ 0.00023716]] [[-0.09227223 -0.09203717]
 [-0.09203717 -0.09213115]
 [-0.09172777 -0.09196283]
 [-0.09196283 -0.09186885]]
0.01
[[-0.00157101]
 [-0.00186978]
 [ 0.00051829]
 [ 0.00024021]] [[-0.09327519 -0.09303758]
 [-0.09303758 -0.09313257]
 [-0.09272

[[0.0019154 ]
 [0.00159651]
 [0.0024537 ]
 [0.00214163]] [[-0.03617655 -0.03603203]
 [-0.03603203 -0.03611077]
 [-0.03582345 -0.03596797]
 [-0.03596797 -0.03588923]]
0.01
[[0.00196868]
 [0.00164093]
 [0.00252178]
 [0.00220105]] [[-0.03718145 -0.03703292]
 [-0.03703292 -0.03711385]
 [-0.03681855 -0.03696708]
 [-0.03696708 -0.03688615]]
0.01
[[0.00202196]
 [0.00168535]
 [0.00258986]
 [0.00226046]] [[-0.03818635 -0.03803381]
 [-0.03803381 -0.03811693]
 [-0.03781365 -0.03796619]
 [-0.03796619 -0.03788307]]
0.01
[[0.00207524]
 [0.00172978]
 [0.00265794]
 [0.00231988]] [[-0.03919125 -0.0390347 ]
 [-0.0390347  -0.03912   ]
 [-0.03880875 -0.0389653 ]
 [-0.0389653  -0.03888   ]]
0.01
[[0.00212852]
 [0.0017742 ]
 [0.00272602]
 [0.00237929]] [[-0.04019616 -0.04003559]
 [-0.04003559 -0.04012308]
 [-0.03980384 -0.03996441]
 [-0.03996441 -0.03987692]]
0.01
[[0.0021818 ]
 [0.00181862]
 [0.00279411]
 [0.00243871]] [[-0.04120106 -0.04103647]
 [-0.04103647 -0.04112615]
 [-0.04079894 -0.04096353]
 [-0.04

[[0.00446936]
 [0.00372519]
 [0.00572519]
 [0.00499714]] [[-0.08441174 -0.08407468]
 [-0.08407468 -0.08425839]
 [-0.08358826 -0.08392532]
 [-0.08392532 -0.08374161]]
0.01
[[0.00452242]
 [0.00376939]
 [0.00579349]
 [0.00505678]] [[-0.08541664 -0.08507556]
 [-0.08507556 -0.08526146]
 [-0.08458336 -0.08492444]
 [-0.08492444 -0.08473854]]
0.01
[[0.00457548]
 [0.00381358]
 [0.0058618 ]
 [0.00511642]] [[-0.08642154 -0.08607645]
 [-0.08607645 -0.08626454]
 [-0.08557846 -0.08592355]
 [-0.08592355 -0.08573546]]
0.01
[[0.00462853]
 [0.00385777]
 [0.00593011]
 [0.00517607]] [[-0.08742644 -0.08707734]
 [-0.08707734 -0.08726761]
 [-0.08657356 -0.08692266]
 [-0.08692266 -0.08673239]]
0.01
[[0.00468156]
 [0.00390195]
 [0.00599844]
 [0.00523573]] [[-0.08843133 -0.08807823]
 [-0.08807823 -0.08827068]
 [-0.08756867 -0.08792177]
 [-0.08792177 -0.08772932]]
0.01
[[0.00473459]
 [0.00394611]
 [0.00606677]
 [0.0052954 ]] [[-0.08943623 -0.08907912]
 [-0.08907912 -0.08927376]
 [-0.08856377 -0.08892088]
 [-0.08

[[0.00131289]
 [0.00112938]
 [0.0020831 ]
 [0.00189948]] [[-0.03207242 -0.03202203]
 [-0.03202203 -0.03205223]
 [-0.03192758 -0.03197797]
 [-0.03197797 -0.03194777]]
0.01
[[0.00135383]
 [0.00116458]
 [0.00214829]
 [0.00195892]] [[-0.03307469 -0.03302272]
 [-0.03302272 -0.03305386]
 [-0.03292531 -0.03297728]
 [-0.03297728 -0.03294614]]
0.01
[[0.00139476]
 [0.00119978]
 [0.00221348]
 [0.00201838]] [[-0.03407695 -0.03402341]
 [-0.03402341 -0.03405549]
 [-0.03392305 -0.03397659]
 [-0.03397659 -0.03394451]]
0.01
[[0.00143569]
 [0.00123498]
 [0.00227867]
 [0.00207783]] [[-0.03507921 -0.03502409]
 [-0.03502409 -0.03505712]
 [-0.03492079 -0.03497591]
 [-0.03497591 -0.03494288]]
0.01
[[0.00147662]
 [0.00127017]
 [0.00234387]
 [0.0021373 ]] [[-0.03608148 -0.03602478]
 [-0.03602478 -0.03605875]
 [-0.03591852 -0.03597522]
 [-0.03597522 -0.03594125]]
0.01
[[0.00151754]
 [0.00130535]
 [0.00240908]
 [0.00219676]] [[-0.03708374 -0.03702547]
 [-0.03702547 -0.03706038]
 [-0.03691626 -0.03697453]
 [-0.03

[[0.0032691 ]
 [0.00281027]
 [0.00522087]
 [0.00476187]] [[-0.08018102 -0.08005505]
 [-0.08005505 -0.08013054]
 [-0.07981898 -0.07994495]
 [-0.07994495 -0.07986946]]
0.01
[[0.00330957]
 [0.002845  ]
 [0.00528653]
 [0.00482179]] [[-0.08118328 -0.08105574]
 [-0.08105574 -0.08113217]
 [-0.08081672 -0.08094426]
 [-0.08094426 -0.08086783]]
0.01
[[0.00335002]
 [0.00287971]
 [0.0053522 ]
 [0.00488173]] [[-0.08218554 -0.08205643]
 [-0.08205643 -0.0821338 ]
 [-0.08181446 -0.08194357]
 [-0.08194357 -0.0818662 ]]
0.01
[[0.00339045]
 [0.00291441]
 [0.00541789]
 [0.00494168]] [[-0.0831878  -0.08305711]
 [-0.08305711 -0.08313543]
 [-0.0828122  -0.08294289]
 [-0.08294289 -0.08286457]]
0.01
[[0.00343087]
 [0.00294909]
 [0.0054836 ]
 [0.00500165]] [[-0.08419007 -0.0840578 ]
 [-0.0840578  -0.08413706]
 [-0.08380993 -0.0839422 ]
 [-0.0839422  -0.08386294]]
0.01
[[0.00347127]
 [0.00298376]
 [0.00554932]
 [0.00506164]] [[-0.08519233 -0.08505849]
 [-0.08505849 -0.0851387 ]
 [-0.08480767 -0.08494151]
 [-0.08

# Backtesting

In [7]:
# Backtesting: Clean the result and calculate the returns
returns_result = data_test.copy().iloc[train_window:,range(N)]

# Create columns
for i in range(N):
    returns_result['money in '+str(i)] = 0.5
returns_result['risk-free asset value'] = 1
returns_result['Portfolio Value'] = 0

# Some variables for convenience
I = np.identity(N)
In = np.concatenate((I,-I))
x = []
x_before_return = []
for t in range(len(returns_result.index)):
    x.append(np.array([[1/N]]*N))
    x_before_return.append(np.array([[1/N]]*N))

for t in range(len(returns_result.index)):
    j = int(t/train_window)
    k = t - j * train_window
    
    # Write the money invested to the testing dataset
    if (k>0):
        x_before_return[t]=x[t-1]+np.dot(In.T, A0_list_test[j][k-1]+\
                                         np.dot(A1_list_test[j][k-1],\
                                                x[t-1]))
        for i in range(N): 
            x[t][i] = (1+returns_result.iloc[t,i]) * \
                      x_before_return[t][i]
            returns_result.loc[returns_result.index[t],\
                               'money in '+str(i)] = x[t][i]
    # Calculate portfolio values and risk-free asset value
    if (t==0):
        returns_result.loc[returns_result.index[t],\
                           'Portfolio Value'] = 1
        returns_result.loc[returns_result.index[t],\
                           'risk-free asset value'] = 0
    else:
        returns_result.loc[returns_result.index[t],'risk-free asset value'] = \
            (returns_result.loc[returns_result.index[t-1],'Portfolio Value']-x_before_return[t].sum())*(1+rf)
        returns_result.loc[returns_result.index[t],'Portfolio Value'] = \
            x[t].sum() + returns_result.loc[returns_result.index[t],'risk-free asset value']

# Calculate daily return
returns_result['Daily return'] =  returns_result['Portfolio Value']/\
                            returns_result['Portfolio Value'].shift(1)
# Calculate allocation weights
for i in range(N):
    returns_result['w'+str(i)] = returns_result['money in '+str(i)]/\
                            returns_result['Portfolio Value']


In [8]:
returns_result.head()

,AAPL,GOOG,money in 0,money in 1,risk-free asset value,Portfolio Value,Daily return,w0,w1
Date,,,,,,,,,
2018-04-12,0.009859,0.012295,0.500000,0.500000,0.000000,1.000000,NaN,0.500000,0.500000
2018-04-13,0.003388,-0.003138,0.502318,0.499146,-0.001339,1.000125,1.000125,0.502255,0.499083
2018-04-16,0.006238,0.008462,0.506090,0.504107,-0.002705,1.007492,1.007367,0.502326,0.500358
2018-04-17,0.013764,0.034856,0.513625,0.522297,-0.003865,1.032058,1.024383,0.497671,0.506073
2018-04-18,-0.002244,-0.001936,0.513128,0.522048,-0.005285,1.029890,0.997900,0.498235,0.506896


In [9]:
returns_result.tail()

,AAPL,GOOG,money in 0,money in 1,risk-free asset value,Portfolio Value,Daily return,w0,w1
Date,,,,,,,,,
2019-07-08,-0.020614,-0.013468,0.536922,0.492406,0.192565,1.221893,0.985472,0.439418,0.402986
2019-07-09,0.006099,0.007596,0.540748,0.496771,0.191408,1.228927,1.005756,0.440016,0.404232
2019-07-10,0.009889,0.013913,0.546534,0.504222,0.190452,1.241208,1.009993,0.440325,0.406235
2019-07-11,-0.007282,0.003271,0.543087,0.506476,0.189323,1.238885,0.998129,0.438367,0.408816
2019-07-12,0.007683,0.000603,0.544460,0.504078,0.194813,1.243351,1.003604,0.437898,0.405419


In [10]:
returns_result.to_csv('Continuous G-learning.csv')

In [ ]:
#######################################################################